In [1]:
library(igraph, warn.conflicts = F, verbose = F)
library(foreach, warn.conflicts = F, verbose = F)
library(doParallel, warn.conflicts = F, verbose = F)
library(purrr, warn.conflicts = F, verbose = F)
library(ggplot2)

Loading required package: iterators

Loading required package: parallel



In [2]:
totalCores = detectCores()

cluster <- makeCluster(totalCores[1]-1) 
registerDoParallel(cluster)

In [3]:
source('creation_destruction.R')

In [4]:
alphas <- c(0.01, seq(from = 0.1, to = 1, by = 0.3))

In [5]:
n<-50

In [6]:
parms<-list(list(name = "er",
                 type='er',
          n = n, p = 1.1 * log(n)/n,
          destroy_method = "random",
          metric_method = "be"),
     list(name = "er_target",type='er',
          n = n, p = 1.1 * log(n)/n,
          destroy_method = "targeted",
          metric_method = "be"),
     list(name = "conf_k1_g2",type='conf',
          n = n, k_min=1, gamma=2,
          destroy_method = "targeted",
          metric_method = "be"),
     list(name = "conf_k1_g2",type='conf',
          n = n, k_min=1, gamma=2,
          destroy_method = "random",
          metric_method = "be"),
    list(name = "conf_k1_g3",type='conf',
          n = n, k_min=1, gamma=3,
          destroy_method = "targeted",
          metric_method = "be"),
     list(name = "conf_k1_g3",type='conf',
          n = n, k_min=1, gamma=3,
          destroy_method = "random",
          metric_method = "be"),
     list(name = "conf_k2_g2",type='conf',
          n = n, k_min=2, gamma=2,
          destroy_method = "targeted",
          metric_method = "be"),
     list(name = "conf_k2_g2",type='conf',
          n = n, k_min=2, gamma=2,
          destroy_method = "random",
          metric_method = "be"),
     list(name = "conf_k2_g3",type='conf',
          n = n, k_min=2, gamma=3,
          destroy_method = "targeted",
          metric_method = "be"),
     list(name = "conf_k2_g3",type='conf',
          n = n, k_min=2, gamma=3,
          destroy_method = "random",
          metric_method = "be"),
     list(name = "homo",type='homo',
          n = n, k_min=3,
          destroy_method = "random",
          metric_method = "be"),
    list(name = "homo",type='homo',
          n = n, k_min=3,
          destroy_method = "targeted",
          metric_method = "be"))

In [7]:
# pb <- txtProgressBar(min = 0, max = length(parms), style = 3, width = 50, char = "=")

# results <- list()

# for (s in seq_along(parms)) {

#     mean_survived <- vector()
#     std_survived <- vector()
#     for (a in seq_along(alphas)) {

#         survived_alpha <- vector()
#         for (i in 1:10) {
#             g <- generate(method = parms[[s]]$type,
#                           n = parms[[s]]$n,
#                           p = parms[[s]]$p,
#                           k_min = parms[[s]]$k_min,
#                           gamma = parms[[s]]$gamma,
#                           metric_method = parms[[s]]$metric_method,
#                           alpha = alphas[a])
#             survived_alpha[i] <- survived_size(g,
#                                                metric_method = parms[[s]]$metric_method,
#                                                destroy_method =parms[[s]]$destroy_method)
#         }

#         mean_survived[a] <- mean(survived_alpha)
#         std_survived[a] <- sd(survived_alpha)
#     }

#     setTxtProgressBar(pb, s)

#     results[[s]] <- data.frame(alpha=alphas, mean = mean_survived, std = std_survived, name = parms[[s]]$name,
#         destroy_protocol = parms[[s]]$destroy_method, metric_method = parms[[s]]$metric_method)
# }

In [8]:
# pb <- txtProgressBar(min = 0, max = length(parms), style = 3, width = 50,
# char = '=')

results <- foreach(s = seq_along(parms), .combine = rbind) %dopar% {
    library(igraph)
    source('creation_destruction.R')

    mean_survived <- vector()
    std_survived <- vector()
    for (a in seq_along(alphas)) {

        survived_alpha <- vector()
        for (i in 1:10) {
            g <- generate(method = parms[[s]]$type, n = parms[[s]]$n, p = parms[[s]]$p,
                k_min = parms[[s]]$k_min, gamma = parms[[s]]$gamma, metric_method = parms[[s]]$metric_method,
                alpha = alphas[a])
            survived_alpha[i] <- survived_size(g, metric_method = parms[[s]]$metric_method,
                destroy_method = parms[[s]]$destroy_method)
        }

        mean_survived[a] <- mean(survived_alpha)
        std_survived[a] <- sd(survived_alpha)
    }

    # setTxtProgressBar(pb, s)

    return(data.frame(alpha = alphas, mean = mean_survived, std = std_survived, name = parms[[s]]$name,
        destroy_protocol = parms[[s]]$destroy_method, metric_method = parms[[s]]$metric_method))
}

In [9]:
results

alpha,mean,std,name,destroy_protocol,metric_method
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
0.01,0.462,0.19077619,er,random,be
0.10,0.654,0.23552306,er,random,be
0.40,0.952,0.04638007,er,random,be
0.70,0.974,0.01897367,er,random,be
1.00,0.970,0.01943651,er,random,be
0.01,0.322,0.10432854,er_target,targeted,be
0.10,0.426,0.06257440,er_target,targeted,be
0.40,0.676,0.16886550,er_target,targeted,be
0.70,0.932,0.04237400,er_target,targeted,be
